In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%matplotlib inline 

In [ ]:
from yaml import load as yaml_load
import findspark
findspark.init()
import pyspark
from pyspark.sql import SparkSession
from pyspark.ml.feature import MinMaxScaler
from pyspark.ml.feature import VectorAssembler
import pyspark.sql.functions as func
spark = SparkSession.builder.master("local").appName("Data cleaning").getOrCreate()

In [ ]:
import os
import sys
sys.path.append('../')

In [ ]:
sc = spark.sparkContext

In [ ]:
import json

In [ ]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config

def normalize(df, columns):
    """
    
    """
    aggExpr = []
    for column in columns:
        aggExpr.append(mean(df[column]).alias(column))
    averages = df.agg(*aggExpr).collect()[0]
    selectExpr = []
    for column in columns:
        selectExpr.append(df[column] - averages[column])
    return df.select(selectExpr)

def get_dummies(df, list_columns):
    """
    
    """
    
    def join_all(dfs, keys):
        if len(dfs) > 1:
            return dfs[0].join(join_all(dfs[1:], keys), on = keys, how = 'inner')
        else:
            return dfs[0]
    dfs = []
    combined = []
    pivot_cols = list_columns
    keys = df.columns
    
    for pivot_col in pivot_cols:
        pivotDF =  df.groupBy(keys).pivot(pivot_col).count()
        new_names = pivotDF.columns[:len(keys)] +  ["{0}_{1}".format(pivot_col, c)\
                                                for c in pivotDF.columns[len(keys):]] 
        df = pivotDF.toDF(*new_names).fillna(0)
        combined.append(df)
    df_result =  join_all(combined, keys)
    
    return df_result

In [ ]:
config_file = "/home/ml/Documents/crimes_chigaco/config/config.yml"
config = _build_configuration(config_file)

In [ ]:
def path_crime():
    """
    :return:
    """
    return config['connect']['PathCrimes']

def path_socio():
    """
    :return:
    """
    return config['connect']['PathSocioEco']

def path_columns():
    """
    :return:
    """
    return config['connect']['Pathcolumns']

def path_temperature():
    """
    :return:
    """

    return config['connect']['PathTemperature']

def path_sky():
    """
    :return:
    """
    return config['connect']['PathSky']
def spark_shape(self):
    return (self.count(), len(self.columns))
pyspark.sql.dataframe.DataFrame.shape = spark_shape

In [ ]:
column_name = json.loads(open(path_columns()).read())

In [ ]:
def df_crime():
    """
    :return:
    """
    column_name = json.loads(open(path_columns()).read())
    df_crimes = spark.read.format("csv").option("header","true").\
    option("mode","DROPMALFORMED").option("delimiter", ";").load(path_crime())
    for old_name, new_name in column_name['DataCrimes'].items():
        df_crimes = df_crimes.withColumnRenamed(old_name, new_name)
    df_crimes = df_crimes.withColumn("date", func.to_timestamp("date", "MM/dd/yyyy hh:mm:ss aaa"))
    
    if config["List_of_crimes_prediction"]["with_merge"]:
        df_crimes = df_crimes.na.replace(config["List_of_crimes_prediction"]["to_merge"],'primary_type')
        return df_crimes

    else:
        return df_crimes

In [ ]:
def df_socio():
    """

    :return:
    """
    column_name = json.loads(open(path_columns()).read())
    df_socio = spark.read.format("csv").option("header","true").\
    option("mode","DROPMALFORMED").option("delimiter", ",").load(path_socio())
    for old_name, new_name in column_name['SocioEco'].items():
        df_socio = df_socio.withColumnRenamed(old_name, new_name)
        
    column_names_to_normalize = ['pct_housing_crowded','pct_households_below_poverty',  'pct_age16_unemployed' , 'pct_age25_no_highschool', 'pct_not_working_age','per_capita_income',
                'hardship_index']

    return df_socio

In [ ]:
df_socio_ = df_socio()

In [ ]:
df_socio_.toPandas()

In [ ]:
def df_nb_crimes_spark():
    
    """
    
    """
    df_S = df_socio()
    df_C = df_crime()
    df_C = df_C.filter(func.col('primary_type').isin(config['NameCrime']))
    df_C = df_C.filter((func.col('date') > '2015-03-18') & (func.col('date') < '2017-01-15')).withColumn("month", func.month(func.col("date"))).withColumn("year", func.year(func.col("date"))).groupBy('community_area_number', 'month', 'year', 'primary_type').agg(func.count(df_C.id).alias('nb_crimes')).join(df_S, ['community_area_number'], "inner")
    df_C = get_dummies(df_C , list_columns=['primary_type', 'community_area_name', 'month'])
    return df_C

In [ ]:
%%time
df_nb_crimes_spark_ = df_nb_crimes_spark()      

In [ ]:
df_nb_crimes_spark_.limit(1000).toPandas().sample(10)

In [ ]:
def df_temperature_spark():
    
    """
    :return:
    """
    # pd.options.mode.chained_assignment = None
    df = spark.read.format("csv").option("header","true").\
    option("mode","DROPMALFORMED").option("delimiter", ",").load(path_temperature())
    df = df.select('Chicago', 'datetime').withColumnRenamed('Chicago','Temperature')
    df = df.filter((func.col('datetime') > '2017-05-10') & (func.col('datetime') < '2017-05-15'))
    df = df.withColumn("month", func.month(func.col("datetime"))).\
    withColumn("year", func.year(func.col("datetime"))).withColumn("day", func.dayofmonth(func.col("datetime"))).\
    withColumn("hour", func.hour(func.col("datetime")))
    return df

In [ ]:
df_temp = df_temperature_spark()

In [ ]:
def df_sky():
    """
    :return:
    """
    
    df = spark.read.format("csv").option("header","true").\
    option("mode","DROPMALFORMED").option("delimiter", ",").load(path_sky())
    df = df.select('Chicago', 'datetime')
    df= df.filter((func.col('datetime') > '2017-05-10') & (func.col('datetime') < '2017-05-15'))
    df = get_dummies(df, list_columns=['Chicago'])
    df= df.withColumn("month", func.month(func.col("datetime"))).\
    withColumn("year", func.year(func.col("datetime"))).withColumn("day", func.dayofmonth(func.col("datetime"))).\
    withColumn("hour", func.hour(func.col("datetime")))
  
    
    return df

In [ ]:
df_sky_ = df_sky()

In [ ]:
df_sky_.toPandas().sample(5)

In [ ]:
df_temp.toPandas().sample(5)

In [ ]:
def _load_config_file(config_file):
    """
    Load configuration file
    :param config_file: is the configuration file
    :return: configuration
    :rtype: dict
    """
    with open(config_file) as yml_config:
        return yaml_load(yml_config)

def _build_configuration(config_file):
    """
    Build the operation configuration dict
    :param config_file: is the path to the yaml config_file
    :type: string
    :return: config: global configuration
    :rtype dict
    """
    # yaml config
    config = _load_config_file(config_file)
    return config


In [ ]:
from src.data.make_dataset import LoadDataframe

In [ ]:
config_file = "/home/ml/Documents/crimes_chigaco_spark/config/config.yml"
config = _build_configuration(config_file)

In [ ]:
%%time
obj_df_loaded = LoadDataframe(config, '2013', '2014')

In [ ]:
%%time
df_crime_ = obj_df_loaded.df_crime()

In [ ]:
df_crime_.limit(1000).toPandas().sample(5)

In [ ]:
df_socio_ = obj_df_loaded.df_socio()

In [ ]:
df_socio_.toPandas()

In [ ]:
df_nb_crimes = obj_df_loaded.df_nb_crimes()

In [ ]:
df_nb_crimes.limit(1000).toPandas().sample(5)

In [ ]:
df_nb_crimes.columns

In [ ]:
columns_feature = ['community_area_number',
 'month',
 'year',
 'primary_type',
 'nb_crimes',
 'community_area_name',
 'pct_housing_crowded',
 'pct_households_below_poverty',
 'pct_age16_unemployed',
 'pct_age25_no_highschool',
 'pct_not_working_age',
 'per_capita_income',
 'hardship_index', ]

In [ ]:
list_columns = df_nb_crimes.columns

In [ ]:
data = df_nb_crimes.drop(*['community_area_number', 'month', 'year', 'primary_type', 'community_area_name'])

In [ ]:
data.limit(5).toPandas()

In [ ]:
data.printSchema()

In [ ]:
from pyspark.ml import Pipeline
from pyspark.ml.regression import LinearRegression
from pyspark.ml.regression import GBTRegressor
from pyspark.ml.feature import VectorIndexer
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml import Pipeline
from pyspark.sql.functions import *

In [ ]:
features_socio = ['pct_housing_crowded',
 'pct_households_below_poverty','pct_age16_unemployed',
 'pct_age25_no_highschool',
 'pct_not_working_age',
 'per_capita_income',
 'hardship_index']

In [ ]:

data = data.withColumnRenamed('nb_crimes', 'label')

In [ ]:
from pyspark.sql.types import DoubleType
for f in features_socio:
    data = data.withColumn(f, data[f].cast(DoubleType()))

In [ ]:
features = data.columns

In [ ]:
features.remove('label')

In [ ]:
vectorAssembler = VectorAssembler(inputCols=features, outputCol="unscaled_features")
standardScaler = StandardScaler(inputCol="unscaled_features", outputCol="features")

In [ ]:
lr = LinearRegression(maxIter=10, regParam=.01)

In [ ]:
gbt = GBTRegressor(featuresCol="features", maxIter=100)

In [ ]:
help(gbt)

In [ ]:
stages = [vectorAssembler, standardScaler, gbt]

In [ ]:
pipeline = Pipeline(stages=stages)

In [ ]:
(training, test) = data.randomSplit([.7, .3])

In [ ]:
training.limit(5).toPandas()

In [ ]:
model = pipeline.fit(training)

In [ ]:
prediction = model.transform(test)

In [ ]:
prediction.limit(10).toPandas()

In [ ]:
df_result = prediction.select(*['label','prediction']).toPandas()

In [ ]:
def visualisation_prediction(y_test, y_pred):
    import matplotlib
    import matplotlib.pyplot as plt
    matplotlib.rc('xtick', labelsize=30) 
    matplotlib.rc('ytick', labelsize=30) 
    fig, ax = plt.subplots(figsize=(50, 40))
    plt.style.use('ggplot')
    plt.plot(y_pred, y_test, 'ro')
    plt.xlabel('Predicted Crime', fontsize = 30)
    plt.ylabel('Actual Crime', fontsize = 30)
    plt.title('Predicted Y (Crimes) to the Actual Y (Crimes)', fontsize = 30)
    ax.plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'k--', lw=4)

In [ ]:
df_result.sample(5)

In [ ]:
visualisation_prediction(df_result['label'], df_result['prediction'])

In [ ]:
evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(prediction)
print("Root Mean Squared Error (RMSE) on test data = %g" % rmse)

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
r2_score(df_result['label'], df_result['prediction'])  